In [1]:
# lets do some multi-class text classification:

import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from math import exp
from sklearn.pipeline import make_pipeline

In [2]:
# make some data:
X_train = ["new york is a hell of a town",
    "new york was originally dutch",
    "the big apple is great",
    "new york is also called the big apple",
    "nyc is nice",
    "people abbreviate new york city as nyc",
    "the capital of great britain is london",
    "london is in the uk",
    "london is in england",
    "london is in great britain",
    "it rains a lot in london",
    "london hosts the british museum",
    "new york is great and so is london",
    "i like london better than new york"]

y_train = [['New York'],['New York'],['New York'],['New York'],['New York'],['New York'],['London'],
           ['London'],['London'],['London'],['London'],['London'],['New York','London'],['New York','London']]

print(type(X_train))
print(type(y_train))

<class 'list'>
<class 'list'>


In [3]:
# one-hot encode labels:
binarizer = MultiLabelBinarizer()
y_train = binarizer.fit_transform(y_train)
y_train

array([[0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [0, 1],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 0],
       [1, 1],
       [1, 1]])

In [4]:
# train a classifier:
vectorizer = TfidfVectorizer(ngram_range=(1, 2), stop_words='english', max_features = 1000)
classifier = OneVsRestClassifier(LinearSVC(verbose=1, max_iter=100), n_jobs=2)
pipeline =Pipeline([('vectorizer', vectorizer), ('classifier', classifier)])
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=1000, min_df=1,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=Tr...lti_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=1),
          n_jobs=2))])

* http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html
* http://scikit-learn.org/stable/modules/generated/sklearn.svm.LinearSVC.html

In [5]:
# apply:
X_test = ['nice day in nyc', 'welcome to london', 'hello welcome to new york. enjoy it here and london too']
predicted = pipeline.predict(X_test)
predicted

array([[0, 1],
       [1, 0],
       [1, 1]])

In [6]:
# which is:
for item, labels in zip(X_test, binarizer.inverse_transform(predicted)):
    print('%s => %s' % (item, labels))

nice day in nyc => ('New York',)
welcome to london => ('London',)
hello welcome to new york. enjoy it here and london too => ('London', 'New York')


![](http://static.tumblr.com/c5a260708065789ef41ed2c4b9775b0c/hqljtju/Gg1nx44lz/tumblr_static_eqvnfxglppck44wg8o4ckk0go_2048_v2.png)

In [7]:
import eli5

C:\Program Files\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [8]:
# lets see some weights: 
# estimator coef_ hack, see https://github.com/TeamHG-Memex/eli5/issues/106
setattr(classifier.estimator, 'classes_', classifier.classes_)
setattr(classifier.estimator, 'coef_', classifier.coef_)
setattr(classifier.estimator, 'intercept_', classifier.intercept_)
eli5.show_weights(classifier.estimator, vec=vectorizer, target_names=binarizer.classes_)

In [9]:
# explain predictios:
eli5.show_prediction(classifier, X_test[0], vec=vectorizer, target_names=binarizer.classes_)

In [10]:
eli5.show_prediction(classifier, X_test[2], vec=vectorizer, target_names=binarizer.classes_)

In [11]:
# so what are these numbers:
pipeline.decision_function(X_test)

array([[-0.56187798,  0.59797212],
       [ 1.07415119, -0.6376566 ],
       [ 0.18872364,  0.66954439]])

In [12]:
# or in terms of "probability":
sigmoid = np.vectorize(lambda x: 1 / (1 + exp(-x)))
sigmoid(pipeline.decision_function(X_test))

array([[ 0.36311304,  0.64519222],
       [ 0.74538556,  0.34577646],
       [ 0.54704137,  0.66140113]])

Further see:
- Basics: http://www.dataschool.io/machine-learning-with-scikit-learn
- Great book: http://shop.oreilly.com/product/0636920052289.do
- Eli5: https://eli5.readthedocs.io/en/latest/tutorials/sklearn-text.html
- Advanced pipelines:
    - http://scikit-learn.org/stable/auto_examples/hetero_feature_union.html
    - https://signal-to-noise.xyz/post/sklearn-pipeline/

Somewhere on the Internet:
> * Want to impress investors, say you work in **AI**.
> * Want to impress employers, say you work in **data science**.
> * Want to impress hipsters, say you work in **machine intelligence**.
> * Want to impress IBM recruiters, say you work in **cognitive computing**.
> * Don't want to be laughed at by your peers, say you **program some machine learning**.
> * Whatever you say, **from sklearn import linear_model**.